# Classification with Markov Models
*Curtis Miller*

Our objective is to perform classification using a Markov model. We imagine that transitioning between letters in a language occurs according to a Markov chain; that is, the next letter depends only on the current letter in the process generating the text (an extremely unrealistic assumption that works remarkably well sometimes). We imagine that texts were generated by different processes with different chains. To classify, we estimate a transition matrix for each label in our training data. When making predictions, we take a transition matrix for a text we wish to make a prediction for. Then we see which label's transition matrix would lead to a smaller entropy score. The label that minimizes the entropy score is the predicted label.

Below I demonstrate this procedure, attempting to predict the gender of authors of given texts.

In [ ]:
from nltk import bigrams
import numpy as np
import pandas as pd

In [ ]:
def markov_matrix(textstring):
    """Turns a string into a transition matrix of a Markov chain (although rows could sum to zero;
       thus this is not technically a Markov transition matrix if the string is too small)"""
    
    textstring = textstring.lower()
    if textstring[0] != ' ':
        textstring = ' ' + textstring
    if textstring[-1] != ' ':
        textstring += ' '
    chars = " abcdefghijklmnopqrstuvwxyz"
    textstring = ''.join(a for a in textstring if a in chars)
    mat = pd.DataFrame(np.zeros((len(chars),) * 2), index=list(chars), columns=list(chars))
    freqs = pd.Series([u[0] + u[1] for u in bigrams(textstring)]).value_counts()
    freqs.index = pd.MultiIndex.from_tuples([tuple(u) for u in freqs.index])
    mat = mat + freqs.unstack().fillna(0)
    mat = (mat.T / np.maximum(mat.T.sum(), 1)).T.fillna(0)
    return mat

In [ ]:
mat = markov_matrix("The quick brown fox jumped over the lazy sleeping dog.")

mat

In [ ]:
mat.sum(axis=1)

In [ ]:
def getEntropy(text, test):
    """This function gets the entropy of a text's letter transition frequencies in
       relation to a probability matrix of letter transitions from an author. text
       is the frequency matrix from the text to be tested, and test is the author's
       letter transition probability matrix."""
    test = test.copy()
    test[test == 0] = 1
    test = np.log(test)
    
    entrop = text * test
    entrop = -entrop.sum().sum()
    
    return entrop

Now we get a training and test set.

In [ ]:
# Text names

base_dir = "books/"

# Training data
male_text_names = ['LewisCarrollAlicesAdventuresinWonderland.txt',
                   'LewisCarrollATangledTale.txt',
                   'LewisCarrollEightorNineWordsAboutLetterWriting.txt',
                   'LewisCarrollFeedingtheMind.txt',
                   'LewisCarrollSylvieandBruno.txt',
                   'LewisCarrollThroughtheLookingGlass.txt',
                   'CharlesDickensAChildsDreamofaStar.txt',
                   'CharlesDickensAChildsHistoryofEngland.txt',
                   'CharlesDickensAChristmasCarol.txt',
                   'CharlesDickensATaleofTwoCities.txt',
                   'CharlesDickensBarnabyRudge.txt',
                   'CharlesDickensBleakHouse.txt',
                   'CharlesDickensCaptainBoldheart.txt',
                   'CharlesDickensChildrensStories.txt',
                   'CharlesDickensDavidCopperfield.txt',
                   'CharlesDickensDoctorMarigold.txt',
                   'CharlesDickensDombeyandSon.txt',
                   'CharlesDickensGeorgeSilvermansExplanation.txt',
                   'CharlesDickensGoingIntoSociety.txt',
                   'CharlesDickensGreatExpectations.txt',
                   'CharlesDickensHardTimes.txt',
                   'CharlesDickensOliverTwist.txt',
                   'CharlesDickensStoriesAboutChildrenEveryChildCanRead.txt',
                   'CharlesDickensTheBattleofLife.txt',
                   'CharlesDickensTheChimes.txt',
                   'CharlesDickensTheCricketontheHearth.txt',
                   'ArthurConanDoyleSirNigel.txt',
                   'ArthurConanDoyleTheAdventureoftheBruceParingtonPlan.txt',
                   'ArthurConanDoyleTheAdventureoftheCardboardBox.txt',
                   'ArthurConanDoyleTheAdventureoftheDevilsFoot.txt',
                   'ArthurConanDoyleTheAdventureoftheDyingDetective.txt',
                   'ArthurConanDoyleTheAdventureoftheRedCircle.txt',
                   'ArthurConanDoyleTheAdventureofWisteriaLodge.txt',
                   'ArthurConanDoyleTheAdventuresofSherlockHolmes.txt',
                   'ArthurConanDoyleTheCrimeoftheCongo.txt',
                   'ArthurConanDoyleTheGreatBoerWar.txt',
                   'ArthurConanDoyleTheHoundoftheBaskervilles.txt',
                   'ArthurConanDoyleTheMemoirsofSherlockHolmes.txt',
                   'ArthurConanDoyleTheReturnofSherlockHolmes.txt',
                   'ArthurConanDoyleUncleBernacTheMemoryoftheEmpire.txt',
                   'OscarWildeAHouseofPomegranates.txt',
                   'OscarWildeChildreninPrison.txt',
                   'OscarWildeDeProfundis.txt',
                   'OscarWildeEssaysandLectures.txt',
                   'OscarWildeImpressionsofAmerica.txt',
                   'OscarWildeTheCantervilleGhost.txt',
                   'OscarWildeTheHappyPrince.txt',
                   'OscarWildeThePictureofDorianGrey.txt',
                   'JamesJoyceUlysses.txt',
                   'JamesJoyceDubliners.txt',
                   'HermanMelvilleMobyDick.txt',
                   'ArthurConanDoyleTheAdventuresofGerard.txt']

# Training data
female_text_names = ['MaryShelleyFrankenstein.txt',
                     'MaryShelleyMathilda.txt',
                     'MaryShelleyTheLastMan.txt',
                     'AgathaChristieTheMysteriousAffairatStyles.txt',
                     'AgathaChristieTheSecretAdversary.txt',
                     'AynRandAnthem.txt',
                     'KateChopinTheAwakeningandotherstories.txt',
                     'KateChopinBayouFolk.txt',
                     'KateChopinAtFault.txt',
                     'LucyMaudMontgomeryAnneofAvonlea.txt',
                     'LucyMaudMontgomeryAnneofGreenGables.txt',
                     'LucyMaudMontgomeryAnneoftheIsland.txt',
                     'LucyMaudMontgomeryAnnesHouseofDreams.txt',
                     'LucyMaudMontgomeryChroniclesofAvonlea.txt',
                     'LucyMaudMontgomeryFurtherChroniclesofAvonlea.txt',
                     'LucyMaudMontgomeryKilmenyoftheOrchard.txt',
                     'LucyMaudMontgomeryRainbowValley.txt',
                     'LucyMaudMontgomeryRillaofIngleside.txt',
                     'LucyMaudMontgomeryShortStories1904.txt',
                     'LucyMaudMontgomeryShortStories18961901.txt',
                     'LucyMaudMontgomeryShortStories19051906.txt',
                     'LucyMaudMontgomeryShortStories19071908.txt',
                     'LucyMaudMontgomeryShortStories19091922.txt',
                     'LucyMaudMontgomeryTheGoldenRoad.txt',
                     'LucyMaudMontgomeryTheStoryGirl.txt',
                     'EdithWhartonAutresTemps.txt',
                     'EdithWhartonBunnerSisters.txt',
                     'EdithWhartonComingHome.txt',
                     'EdithWhartonCrucialInstances.txt',
                     'EdithWhartonEthanFrome.txt',
                     'EdithWhartonFightingFrance.txt',
                     'EdithWhartonKerfol.txt',
                     'EdithWhartonMadamedeTreymes.txt',
                     'EdithWhartonSanctuary.txt',
                     'EdithWhartonSummer.txt',
                     'EdithWhartonTalesofMenandGhosts.txt',
                     'EdithWhartonTheAgeofInnocence.txt',
                     'EdithWhartonTheChoice.txt',
                     'EdithWhartonTheCustomoftheCountry.txt',
                     'EdithWhartonTheDescentofManandOtherStories.txt',
                     'EdithWhartonTheEarlyShortFiction1.txt',
                     'EdithWhartonTheEarlyShortFiction2.txt',
                     'EdithWhartonTheFruitoftheTree.txt',
                     'EdithWhartonTheGlimpsesoftheMoon.txt',
                     'EdithWhartonTheGreaterInclination.txt',
                     'EdithWhartonTheHermitandtheWildWomanandothers.txt',
                     'EdithWhartonTheHouseofMirth.txt',
                     'EdithWhartonTheLongRun.txt',
                     'EdithWhartonTheMarne.txt',
                     'EdithWhartonTheReef.txt',
                     'EdithWhartonTheTouchstone.txt',
                     'EdithWhartonTheTriumphofNight.txt',
                     'EdithWhartonTheValleyofDecision.txt',
                     'EdithWhartonXingu.txt']

# Test data
unknown_text_names = ['JaneAustenPrideandPrejudice.txt',
                      'LouisaMayAlcottLittleWomen.txt',
                      'MarkTwainAdventuresofHuckleberryFinn.txt',
                      'VoltaireMicromegas.txt',
                      'JosephConradHeartofDarkness.txt',
                      'EmilyBronteWutheringHeights.txt',
                      'AnneBronteAgnesGrey.txt',
                      'GeorgeOrwell1984.txt',
                      'GeorgeEliotMiddlemarch.txt',
                      'CharlotteBronteJaneEyre.txt',
                      'AnonymousTheRomanceofLust.txt',
                      'AnonymousForbiddenFruit.txt',
                      'AnonymousLauraMiddleton.txt',
                      'AnonymousBeautyandtheBeast.txt']

We read in transition matrices for all these documents.

In [ ]:
male_mats = list()
female_mats = list()
unknown_mats = list()

for n in male_text_names:
    with open(base_dir + n, encoding="utf8") as f:
        bookstr = f.read()
    male_mats.append(markov_matrix(bookstr))

for n in female_text_names:
    with open(base_dir + n, encoding="utf8") as f:
        bookstr = f.read()
    female_mats.append(markov_matrix(bookstr))

for n in unknown_text_names:
    with open(base_dir + n, encoding="utf8") as f:
        bookstr = f.read()
    unknown_mats.append(markov_matrix(bookstr))

We estimate the transition matrix for different possible labels.

In [ ]:
male_trans = sum(male_mats)
female_trans = sum(female_mats)

# Normalize matrices to 1
male_trans = (male_trans.T / np.maximum(male_trans.T.sum(), 1)).T.fillna(0)
female_trans = (female_trans.T / np.maximum(female_trans.T.sum(), 1)).T.fillna(0)

male_trans

In [ ]:
female_trans

We can now perform prediction.

In [ ]:
lambdamat = pd.DataFrame([{"male": getEntropy(u, male_trans),
                           "female": getEntropy(u, female_trans)} for u in unknown_mats],
                         index=unknown_text_names)
lambdamat

In [ ]:
label = pd.Series(["female"] * len(unknown_mats), index=unknown_text_names)
label[lambdamat.female > lambdamat.male] = "male"
label

It seems that our algorithm has a tendency to predict female instead of male and incorrectly guesses the gender of most authors (except the anonymous ones for which we do not know the gender). There were some normalizing steps we skipped, though.